In [2]:
import numpy as np
import matplotlib.pyplot as plt

# Generating Normal Random Variables


It's important to notice that if $X\sim N(0, 1)$, then

$$\mu + \sigma X\sim N(\mu, \sigma^2)$$

This means that we can focus on methods for generating standard normal random variables and then use a simple linear transformation to obtain arbitrary normal distributions.



## The Box-Muller Algorithm

One of the simplest and most popular methods for generating normal random variables is the Box-Muller algorithm.  This method generates two i.i.d. random variables at once by exploiting some symmetries of the normal distribution in polar coordinates.

Suppose $X, Y\sim N(0, 1)$ are i.i.d. standard normal random variables.  Their joint pdf is therefore

$$f_{XY}(x, y) = \frac{1}{2\pi}e^{-(x^2 + y^2)/2}$$

Now define the new variables $V$ and $W$ such that

$$X = \sqrt{V}\cos W \:\textrm{ and }\: Y = \sqrt{V}\sin W$$

(These are almost polar coordinates, but we are using $V = R^2$ and $W = \Theta$ instead of the usual $R$ and $\Theta$.)

The joint pdf of $V$ and $W$ is therefore

$$f_{VW}(v, w) = \frac{1}{4\pi}e^{-v/2}$$

(You should prove this as an exercise.  Notice that you can't just substitute $x = \sqrt{v}\cos(w)$ and $y = \sqrt{v}\sin(w)$ into $f_{XY}(x, y)$.  You also need to find the Jacobian of the change of variables, which contributes an extra factor of $1/2$.)

This joint distribution factors into

$$f_{VW}(v, w) = f_{V}(v)f_{W}(w) = \frac{1}{2}e^{-v/2}\cdot \frac{1}{2\pi}$$

and so we know that $V$ and $W$ are independent and $V\sim \textrm{Exp}(1/2)$ and $W\sim U(0, 2\pi)$.  The Box-Muller algorithm just uses this idea in reverse:

1) Generate $V\sim \textrm{Exp}(1/2)$ and $W\sim U(0, 2\pi)$.

2) Set $X = \sqrt{V}\cos(W)$ and $Y = \sqrt{V}\sin(W)$.

This method requires two i.i.d. uniform random variables (one to generate $V$ with the inverse transform method and another to generate $W$) and produces two i.i.d. normal random variables.  It therefore only requires one uniform random variable per normal 
random variable.

(This method is reasonably efficient and quite simple to implement, so it is used in many programming libraries.  In particular, this is how numpy generates normally distributed random variables.)

## The Marsaglia-Bray Algorithm

The Box-Muller algorithm requires calculating a sine and a cosine.  On many processors, this is by far the slowest part of the algorithm.  The Marsaglia-Bray algorithm is a modification using acceptance-rejection that avoids the use of trigonometric functions.

The key idea of this algorithm is very similar to the previous version.  Suppose that $x$ and $y$ are uniformly distributed on the unit disc.  That is,

$$f_{XY}(x, y) = \left\{ \begin{array}{cc} 1/\pi & x^2 + y^2 \leq 1 \\ 0 & \textrm{otherwise} \end{array} \right.$$

Now define random variables $U$ and $V$ such that $X = \sqrt{U}\cos(V)$ and $Y = \sqrt{U}\sin(V)$.  The joint pdf of $U$ and $V$ is

$$f_{UV}(u, v) = \left\{ \begin{array}{cc} \frac{1}{2\pi} & (u, v)\in [0, 1]\times [0, 2\pi] \\ 0 & \textrm{otherwise} \end{array} \right.$$

This means that $U$ and $V$ are uniformly distributed on the box $[0, 1]\times [0, 2\pi]$ and therefore $U\sim U(0, 1)$ and $V\sim U(0, 2\pi)$ are independent.

Therefore, if we choose a point $(X, Y)$ uniformly on the unit disc, then its angle $\Theta = \arctan(X/Y)$ will be uniformly distributed on $[0, 2\pi]$, which is what we want for the Box-Muller method, but its radius will also be distributed uniformly, which is not what we want.  We can fix this by using another inverse transform.  If $U = X^2 + Y^2 \sim U(0, 1)$, then $R^2 = -2\ln U\sim \textrm{Exp}(1/2)$.  We therefore have the following algorithm:

1) Generate $(X, Y)$ uniformly on the unit circle.  This is easily accomplished with acceptance-rejection.  We can generate i.i.d. $U_1, U_2\sim U(-1, 1)$ and accept them if $U = U_1^2 + U_2^2 \leq 1$.

2) Set $Z_1 = X\sqrt{-2\ln(U)/U}$ and $Z_2 = Y\sqrt{-2\ln(U)/U}$.

The random variables $Z_1$ and $Z_2$ are i.i.d. standard normal random variables.

The Marsaglia-Bray algorithm avoids computing any sines or cosines, but at the expense of using an acceptance-rejection algorithm, which will often require multiple trials to generate one (pair of) normally distributed value(s).  Whether Box-Muller or Marsaglia-Bray is a better choice will depend on the details of both your problem and your processor.

## Correlated Normal Random Variables

We now have several ways to generate independent normal distributions (with arbitrary mean and variance).  Now we will try to generate non-independent normally distributed random variables.

In general, specifying even two non-independent distributions requires fully specifying the joint distribution.  However, if every linear combination of the $X_i$ is normally distributed (which, in particular, means that the marginal distributions of each $X_i$ are normal), then the joint distribution can be fully specified by giving the means and covariances of all the random variables.  That is, if

$$X = \left(X_1, X_2, \dotsc, X_n\right)^{T}$$

is a random vector where each $Y = \sum_{i=1}^{n}a_iX_i$ is normally distributed (such an $X$ is known as a *multivariate normal* random variable), then we can fully specify the distribution of $X$ by giving a vector $\mu\in\mathbb{R}^{n}$ and a matrix $\Sigma\in\mathbb{R}^{n\times n}$, where

$$\mu_i = \mathbb{E}\left[X_i\right] \:\textrm{ and }\: \Sigma_{ij} = \textrm{Cov}\left[X_i, X_j\right]$$

We write $X\sim N(\mu, \Sigma)$ to denote such a distribution.

The matrix $\Sigma$ is known as the *covariance matrix* of $X$ and it has some important properties.  In particular,

1) The covariance is symmetric.  That is, $\Sigma^{T} = \Sigma$.  This is because $\textrm{Cov}\left[X_i, X_j\right] = \textrm{Cov}\left[X_j, X_i\right]$.

2) The diagonal elements are non-negative.  That is, $\Sigma_{ii} \geq 0$.  This is because $\textrm{Cov}\left[X_i, X_i\right] = \textrm{Var}\left[X_i\right]$.

3) It is positive semi-definite.  That is, $\mathbf{x}^{T}\Sigma\mathbf{x} \geq 0$ for all $\mathbf{x}\in\mathbb{R}^{n}$.  Equivalently, all of the eigenvalues of $\Sigma$ are non-negative.  It's worth noting that $\Sigma$ is positive definite (so that both "non-negative" can be replaced with "positive" in both statements) if and only if it has full rank.

One can show that the if $X$ is a multivariate normal random variable with mean vector $\mu$ and invertible covariance matrix $\Sigma$, then the joint pdf is given by

$$f_{X}(\mathbf{x}) = \frac{1}{(2\pi)^{n/2}|\Sigma|^{1/2}}\exp\left(-\frac{1}{2}\left(\mathbf{x} - \mu\right)^{T}\Sigma^{-1}\left(\mathbf{x} - \mu\right)\right)$$

(If $\Sigma$ is not invertible, then the joint distribution does not have a pdf.  In this case, life becomes much more difficult.)

In addition, linear combinations of multivariate normal random variables are still multivariate normals.  In particular, if $A\in\mathbb{R}^{n\times n}$ and $b\in\mathbb{R}^{n}$, then

$$Y = AX + \mathbf{b}$$

is also a multivariate normal.  In order to specify the distribution of $Y$, we just have to know its mean and covariance matrix.  Fortunately, we know (for any random vector)

$$\begin{aligned}
\mathbb{E}\left[AX + \mathbf{b}\right] &= A\mathbb{E}\left[X\right] + \mathbf{b} \\
\textrm{Cov}\left[AX + \mathbf{b}\right] &= A\textrm{Cov}\left[X\right]A^{T}
\end{aligned}$$

This means that if $Z\sim N(0, I_n)$ (that is, $Z$ is a random vector of $n$ i.i.d. standard normal random variables), then

$$X = \mu + AZ \sim N(\mu, AA^{T})$$

This gives us a way to generate $X\sim N(\mu, \Sigma)$:

1) Generate $Z$ by generating $n$ i.i.d. standard normal random variables (for instance, with Box-Muller).

2) Determine the matrix $A$ such that $AA^{T} = \Sigma$.

3) Set $X = \mu + AZ$.

The only potentially difficult part here is step (2).  Here, we will appeal to some useful theorems from linear algebra.

First, if $\Sigma$ is a symmetric, positive-definite matrix, then it can be written as

$$\Sigma = LDL^{T}$$

where $L$ is a lower triangular matrix and $D$ is a diagonal matrix with non-negative entries.  This means that we can write

$$\begin{aligned}
\Sigma &= \left(L\sqrt{D}\right)\left(\sqrt{D}L^{T}\right) \\
&= \left(L\sqrt{D}\right)\left(L\sqrt{D}\right)^{T}
\end{aligned}$$

where $\sqrt{D}$ is the diagonal matrix whose entries are the square roots of the entries of $D$.  This means that if we set $A = L\sqrt{D}$, then we have

$$\Sigma = AA^{T}$$

### Example

Generate a multivariate normal random vector $X = (X_1, X_2)$ with means and covariance

$$\mu = \begin{pmatrix} 1 \\ 2 \end{pmatrix} \:\textrm{ and }\: \Sigma = \begin{pmatrix} 1 & 0.5 \\ 0.5 & 1.5 \end{pmatrix}$$

In [3]:
import numpy as np

def generate_multivariate_normal(mu, A):
    z = np.random.randn(2).reshape((2, 1))
    return mu + A @ z

mu = np.array([[1], [2]])
Sigma = np.array([[1, 0.5], [0.5, 1.5]])
A = np.linalg.cholesky(Sigma)
x = generate_multivariate_normal(mu, A)

print("X_1 = {}, X_2 = {}".format(x[0,0], x[1,0]))

X_1 = 0.8719976232803976, X_2 = 1.971061935994259
